In [0]:
# This project pulls data from a sample API, organizes it into 3 layers (raw data in bronze, cleaned data in silver, aggregate data in gold), and stores the data as files (csv, xml, delta, parquet, etc.) in their respective layers in the Azure storage container. This process also loads the data into Azure Table Storage (shown only for silver layer data). This is just a sample project to showcase the data pipeline from api call to azure storage, using a serverless approach.

# Created by: Peyton Roehnelt
# Date: 11/8/2025

from azure.storage.blob import BlobServiceClient
import pandas as pd
import requests
from azure.data.tables import TableServiceClient
from azure.core.credentials import AzureSasCredential
import uuid

In [0]:
# Gets client token information, stored as a secret in and Azure scope
sas_token = dbutils.secrets.get(
    scope="test-scope-3",
    key="test-secret"
)

# Connect to Azure Blob Storage using SAS token
blob_service_client = BlobServiceClient(
    account_url="https://peytondatastore.blob.core.windows.net",
    credential=sas_token
)


In [0]:
# API call to mock API, stores in json format
response = requests.get("https://fakerapi.it/api/v2/products?_quantity=10&_taxes=12&_categories_type=uuid")
data = response.json()

In [0]:
# Sample raw data (Bronze layer)
bronze_data = pd.DataFrame(data)

# Clean and transform data (Silver layer)
silver_data = bronze_data[bronze_data["code"] > 10]
gold_data = silver_data.groupby("code").agg({"total":"mean"}).reset_index()

In [0]:
# Ensure storage containers exist before uploading blobs
for container_name in ["bronze", "silver", "gold"]:
    try:
        blob_service_client.create_container(container_name)
    except Exception:
        pass  # Ignore if container already exists

In [0]:
# Function to save each layer to Azure Blob Storage
def upload_df_to_blob(df, container, blob_name):
    csv_data = df.to_csv(index=False)
    blob_client = blob_service_client.get_blob_client(
        container=container, 
        blob=blob_name
    )
    blob_client.upload_blob(csv_data, blob_type='BlockBlob', overwrite=True)

In [0]:
# Def calls to load data to Azure Blob Storage

upload_df_to_blob(bronze_data, "bronze", "product.csv")
upload_df_to_blob(silver_data, "silver", "product.delta")
upload_df_to_blob(gold_data, "gold", "product.csv")

In [0]:
# Use TableServiceClient to connect to Azure Table Storage
sas_token = dbutils.secrets.get(
    scope="test-scope-3",
    key="test-secret"
)
table_service_client = TableServiceClient(
    endpoint="https://peytondatastore.table.core.windows.net",
    credential=AzureSasCredential(sas_token)
)

In [0]:
# Get table from Azure Table Storage
table_client = table_service_client.get_table_client(table_name="silver")

In [0]:
# Reformat data to load to NOSQL Azure Table
def flatten_entity(entity):
    for k, v in entity.items():
        if isinstance(v, dict):
            entity[k] = str(v)
    return entity

entities = []
for idx, row in silver_data.iterrows():
    entity = row.to_dict()
    entity["PartitionKey"] = "default"
    entity["RowKey"] = str(uuid.uuid4())
    entity = flatten_entity(entity)
    entities.append(entity)

# Loads data to Azure Table Storage with upsert (can use create, update, or submit_transaction for different options)
for entity in entities:
    table_client.upsert_entity(entity=entity)


In [0]:
# Read data from Azure Table Storage
entities = list(table_client.list_entities())
df = pd.DataFrame(entities)
display(df)

PartitionKey,RowKey,code,data,locale,status,total
default,17407644-f25e-4c1f-ba4b-fdc8923b5c9a,200,"{'id': 8, 'name': 'Nisi quis quo ex assumenda.', 'description': 'Sequi maxime nihil asperiores non illo. Occaecati ut molestiae non velit. Est et non minus consectetur magni et officia minus.', 'ean': '3242051898244', 'upc': '508732181829', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Aut vitae sit eum laboriosam.', 'description': 'Non repudiandae sed alias quia. Qui occaecati quia quod vel nihil quas. Deleniti magni laborum ut ratione rerum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Itaque et eum voluptate enim.', 'description': 'In vel qui et ipsum quasi. Accusamus ipsa aut atque debitis dolores rem. Est ut vel voluptas repellendus quis autem.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Illum esse et quia veniam.', 'description': 'Rerum dolores aliquam magnam voluptatem labore. Maxime architecto impedit culpa laboriosam soluta. Et rerum aperiam quibusdam recusandae distinctio eum at.', 'url': 'https://picsum.photos/640/480'}], 'net_price': 0.01, 'taxes': 12, 'price': 0.01, 'categories': ['de3f50c0-5916-3513-afba-7f6e10e0358f', 'ceded20b-d08a-3e02-a7ff-9615791e1aeb', '43a63b50-8595-380a-89ca-8b096fc7dff4', '9cfd6ea5-db8a-3ad2-a918-3ddd3e770130', '26b1634c-730d-3cd0-bc1c-0666430e3065', '32464316-0ba3-3565-a421-0b6447473012', '4452ec68-9380-368a-b301-4fd41a0b2db1', '4559cf95-b090-31de-a556-174993d6f5f5'], 'tags': ['vel', 'consequatur', 'sunt']}",en_US,OK,10
default,4d9fbd7a-bbab-4db8-a4b8-bfc1f3a0e832,200,"{'id': 10, 'name': 'Adipisci odit autem rerum.', 'description': 'Impedit delectus odit omnis aspernatur aut. Quia asperiores soluta ut ipsum ipsum fugit animi. Ut autem eos similique inventore corporis temporibus reiciendis nihil. Ex quos error enim harum sunt ea.', 'ean': '0592013787714', 'upc': '421205868699', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Non aliquid quae dolorem et.', 'description': 'Dolore quae aut autem reprehenderit. Sit quia modi qui. Dolorem repudiandae iusto iste dolorum consequuntur ad tempora. Hic sunt voluptates necessitatibus molestiae earum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Minus rerum est debitis.', 'description': 'Esse vel necessitatibus esse saepe rerum voluptatibus dolores autem. Officia error dolor rerum itaque sed qui repellendus. Cumque sit ut alias voluptatum laboriosam qui.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Et et asperiores iste quia.', 'description': 'Laboriosam blanditiis consectetur reprehenderit est cumque eaque totam. Maxime voluptates laborum possimus rem. Mollitia quia exercitationem ipsa incidunt aut ut sunt.', 'url': 'https://picsum.photos/640/480'}], 'net_price': 78322.33, 'taxes': 12, 'price': 87721.01, 'categories': ['d9862cb1-b612-366f-b872-a0933666393a', 'f02bb78a-7a3d-38b1-9f8a-b4bcb8031bc4', '284d0573-43bc-38b4-bbba-02ebdea9e6f9'], 'tags': ['magnam', 'alias', 'nisi', 'aliquid', 'nesciunt', 'voluptatem', 'aut', 'nihil']}",en_US,OK,10
default,532f4d8f-54a5-47b6-8b36-bca4f60a4b49,200,"{'id': 9, 'name': 'Qui optio delectus velit.', 'description': 'Placeat itaque accusantium optio culpa eum veniam enim. Quis corrupti ullam incidunt dicta est odio. Ab sit consequuntur placeat suscipit assumenda.', 'ean': '9521121093012', 'upc': '138392454692', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Consequuntur omnis amet modi.', 'description': 'Nesciunt sapiente totam architecto dolore explicabo. Assumenda corporis atque quia dolorem.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Et et et harum.', 'description': 'Corrupti harum animi consequatur eveniet. Ut et necessitatibus qui alias beatae. Quasi dolorem molestiae temporibus quo cum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Qui sed quam sit dolorem sed.', 'description': 'Repellendus voluptatem similique officia porro quasi maiores. Officia tenetur consectetur et deleniti ratione quia aliqua

In [0]:
# Perform a filter query on Azure Table Storage
filter_query = "code eq 200"
entities = list(
    table_client.query_entities(
        query_filter=filter_query
    )
)
df = pd.DataFrame(entities)
display(df)

PartitionKey,RowKey,code,data,locale,status,total
default,17407644-f25e-4c1f-ba4b-fdc8923b5c9a,200,"{'id': 8, 'name': 'Nisi quis quo ex assumenda.', 'description': 'Sequi maxime nihil asperiores non illo. Occaecati ut molestiae non velit. Est et non minus consectetur magni et officia minus.', 'ean': '3242051898244', 'upc': '508732181829', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Aut vitae sit eum laboriosam.', 'description': 'Non repudiandae sed alias quia. Qui occaecati quia quod vel nihil quas. Deleniti magni laborum ut ratione rerum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Itaque et eum voluptate enim.', 'description': 'In vel qui et ipsum quasi. Accusamus ipsa aut atque debitis dolores rem. Est ut vel voluptas repellendus quis autem.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Illum esse et quia veniam.', 'description': 'Rerum dolores aliquam magnam voluptatem labore. Maxime architecto impedit culpa laboriosam soluta. Et rerum aperiam quibusdam recusandae distinctio eum at.', 'url': 'https://picsum.photos/640/480'}], 'net_price': 0.01, 'taxes': 12, 'price': 0.01, 'categories': ['de3f50c0-5916-3513-afba-7f6e10e0358f', 'ceded20b-d08a-3e02-a7ff-9615791e1aeb', '43a63b50-8595-380a-89ca-8b096fc7dff4', '9cfd6ea5-db8a-3ad2-a918-3ddd3e770130', '26b1634c-730d-3cd0-bc1c-0666430e3065', '32464316-0ba3-3565-a421-0b6447473012', '4452ec68-9380-368a-b301-4fd41a0b2db1', '4559cf95-b090-31de-a556-174993d6f5f5'], 'tags': ['vel', 'consequatur', 'sunt']}",en_US,OK,10
default,4d9fbd7a-bbab-4db8-a4b8-bfc1f3a0e832,200,"{'id': 10, 'name': 'Adipisci odit autem rerum.', 'description': 'Impedit delectus odit omnis aspernatur aut. Quia asperiores soluta ut ipsum ipsum fugit animi. Ut autem eos similique inventore corporis temporibus reiciendis nihil. Ex quos error enim harum sunt ea.', 'ean': '0592013787714', 'upc': '421205868699', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Non aliquid quae dolorem et.', 'description': 'Dolore quae aut autem reprehenderit. Sit quia modi qui. Dolorem repudiandae iusto iste dolorum consequuntur ad tempora. Hic sunt voluptates necessitatibus molestiae earum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Minus rerum est debitis.', 'description': 'Esse vel necessitatibus esse saepe rerum voluptatibus dolores autem. Officia error dolor rerum itaque sed qui repellendus. Cumque sit ut alias voluptatum laboriosam qui.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Et et asperiores iste quia.', 'description': 'Laboriosam blanditiis consectetur reprehenderit est cumque eaque totam. Maxime voluptates laborum possimus rem. Mollitia quia exercitationem ipsa incidunt aut ut sunt.', 'url': 'https://picsum.photos/640/480'}], 'net_price': 78322.33, 'taxes': 12, 'price': 87721.01, 'categories': ['d9862cb1-b612-366f-b872-a0933666393a', 'f02bb78a-7a3d-38b1-9f8a-b4bcb8031bc4', '284d0573-43bc-38b4-bbba-02ebdea9e6f9'], 'tags': ['magnam', 'alias', 'nisi', 'aliquid', 'nesciunt', 'voluptatem', 'aut', 'nihil']}",en_US,OK,10
default,532f4d8f-54a5-47b6-8b36-bca4f60a4b49,200,"{'id': 9, 'name': 'Qui optio delectus velit.', 'description': 'Placeat itaque accusantium optio culpa eum veniam enim. Quis corrupti ullam incidunt dicta est odio. Ab sit consequuntur placeat suscipit assumenda.', 'ean': '9521121093012', 'upc': '138392454692', 'image': 'http://placeimg.com/640/480/tech', 'images': [{'title': 'Consequuntur omnis amet modi.', 'description': 'Nesciunt sapiente totam architecto dolore explicabo. Assumenda corporis atque quia dolorem.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Et et et harum.', 'description': 'Corrupti harum animi consequatur eveniet. Ut et necessitatibus qui alias beatae. Quasi dolorem molestiae temporibus quo cum.', 'url': 'https://picsum.photos/640/480'}, {'title': 'Qui sed quam sit dolorem sed.', 'description': 'Repellendus voluptatem similique officia porro quasi maiores. Officia tenetur consectetur et deleniti ratione quia aliqua